In [2]:
from __future__ import print_function
from rdkit import RDConfig, Chem, Geometry, DistanceGeometry
from rdkit.Chem import ChemicalFeatures, rdDistGeom, Draw, rdMolTransforms
from rdkit.Chem.Pharm3D import Pharmacophore, EmbedLib
from rdkit.Chem.Draw import IPythonConsole, DrawingOptions
from rdkit.Numerics import rdAlignment

import os.path
fdef = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
featFactory = ChemicalFeatures.BuildFeatureFactory(fdef)

In [3]:
moePh43Point = '''#moe:ph4que 2022.5
#pharmacophore 5 tag t value *
scheme t Unified matchsize i 0 title t s $
#feature 4 expr tt color ix x r y r z r r r ebits ix gbits ix
Acc df2f2 64.91299999999998 10.108999999999998 2.6449999999999996 0.9999999999999999 0 300 Don df2f2 64.671 7.668999999999999 3.8289999999999997 0.9999999999999999 0 300 Hyd df2f2 62.08849999999999 11.174999999999997 1.502 0.9999999999999999 0 300 Hyd df2f2 63.903 4.2665 4.58 0.9999999999999999 0 300 
#endpharmacophore'''

In [4]:
#moePh43Point.to_pharmer('abhishek_pharm.json')

In [5]:
moePh43Point.split("\n")

['#moe:ph4que 2022.5',
 '#pharmacophore 5 tag t value *',
 'scheme t Unified matchsize i 0 title t s $',
 '#feature 4 expr tt color ix x r y r z r r r ebits ix gbits ix',
 'Acc df2f2 64.91299999999998 10.108999999999998 2.6449999999999996 0.9999999999999999 0 300 Don df2f2 64.671 7.668999999999999 3.8289999999999997 0.9999999999999999 0 300 Hyd df2f2 62.08849999999999 11.174999999999997 1.502 0.9999999999999999 0 300 Hyd df2f2 63.903 4.2665 4.58 0.9999999999999999 0 300 ',
 '#endpharmacophore']

In [6]:
#########################
# parse the ph4 content #
#########################
def parsePh4Content(moePh4List):
  ph4Info = {}
  header = []
  content = []
  for line in moePh4List.split("\n"):
    if line[0] == "#":
      if len(header) > 0:
        ph4Info[header[0]] = (header,content)
        header = []
        content = []
      header = line.strip().split()
    else:
      content.extend(line.strip().split())
  return ph4Info

############################    
# convert the feature info #
############################
def convertFeatureInfo(ph4InfoDict):
  feats = []
  radii=[]
  # this is the conversion of the moe ph4 nomenclature to the rdkit one
  moePh4Conv = {"Acc":"Acceptor","Don":"Donor","Aro":"Aromatic","Hyd":"Hydrophobe"}  
  header,content = ph4InfoDict['#feature']
  noFeats = int(header[1])
  lenSingleFeature = int((len(header)/2)-1)
  # loop over ph4 features and populate pharmacophore object
  for i in range(noFeats):
    featFamily = content[(lenSingleFeature*i)]
    (x,y,z) = (float(content[(lenSingleFeature*i)+2]),float(content[(lenSingleFeature*i)+3]),
               float(content[(lenSingleFeature*i)+4]))
    r = float(content[(lenSingleFeature*i)+5])
    feats.append(ChemicalFeatures.FreeChemicalFeature(moePh4Conv[featFamily],Geometry.Point3D(x,y,z)))
    radii.append(r)
  return (feats,radii)


In [7]:
def applyRadiiToBounds(radii,pcophore):
  for i in range(len(radii)):
    for j in range(i+1,len(radii)):
      sumRadii = radii[i]+radii[j]
      pcophore.setLowerBound(i,j,max(pcophore.getLowerBound(i,j)-sumRadii,0))
      pcophore.setUpperBound(i,j,pcophore.getUpperBound(i,j)+sumRadii)
#applyRadiiToBounds(radii,pcophore)

In [8]:
from collections import namedtuple
from operator import itemgetter
Match = namedtuple("Match", ["score", "id", "mol"]) # Named tuple to store molecules that match a pharmacophore
def GetTransformMatrix(alignRef,confEmbed,atomMatch):
  alignProbe = []
  for matchIds in atomMatch:
    dummyPoint = Geometry.Point3D(0.0,0.0,0.0)
    for id in matchIds:
      dummyPoint += confEmbed.GetAtomPosition(id)
    dummyPoint /= len(matchIds)
    alignProbe.append(dummyPoint)
  return (rdAlignment.GetAlignmentTransform(alignRef,alignProbe))

def TransformEmbeddings(pcophore,embeddings,atomMatch):
  alignRef = [f.GetPos() for f in pcophore.getFeatures()]
  #print(embeddings)
  SSDs,confs = [],[]
  for embedding in embeddings:
    conf = embedding.GetConformer()
    #print(conf)
    SSD,transformMatrix = GetTransformMatrix(alignRef,conf,atomMatch)
    rdMolTransforms.TransformConformer(conf,transformMatrix)
    SSDs.append(SSD)
    #print(SSDs)
    #confs.append(conf)
    best_fit_index = min(enumerate(SSDs), key=itemgetter(1))[0]
    #print(best_fit_index)
    try:
       mol_id = mol.GetProp("_Name")
    except:
       mol_id = None
    if len(SSDs)>0:
       matched_mol = Match(SSDs[best_fit_index], mol_id, embeddings[best_fit_index])
       #print(matched_mol)
    else:
        print('Length of SSD is zero')
        continue
    #confs.append(matched_mol)
    return(matched_mol)

#a,b=TransformEmbeddings(ligH,pcophore,embeddings,atomMatch)

In [9]:
from operator import itemgetter
ph4Info = parsePh4Content(moePh43Point)
feats,radii = convertFeatureInfo(ph4Info)
pcophore = Pharmacophore.Pharmacophore(feats)
applyRadiiToBounds(radii,pcophore)
molSmiles = ['O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(N(CC)CC)C=C2','O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(N3CCOCC3)C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(OCCC)C=C2','O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(N(C)C)C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(OCC)C=C2','O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(OCC(F)(F)F)C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(C(C)(C)C)C=C2','O=C(NNC1=C(Cl)C=CC=C1Cl)C2=CC=C(N(C)C)C=C2',
             'O=C(NNC1=C(Cl)C=CC=C1Cl)C2=CC=C(OCC)C=C2','O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(CC)C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(C3=CC=CC=C3)C=C2','O=C(NNC1=C(C)C=CC=C1Cl)C2=CC=C(OCC)C=C2',
             'O=C(NNC1=C(Cl)C=CC=C1Cl)C2=CC=C(CCC)C=C2','O=C(NNC1=C(C)C=CC=C1C)C2=CC=C(C(C)(C)C)C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(OCCCC)C=C2','O=C(NNC1=C(Br)C=C(Br)C=C1Br)C2=CC=C(N(C)C)C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(C(OC)=O)C=C2','O=C(NNC1=C(C)C=CC=C1C)C2=CC=C(OCC)C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC(C=CC=C3)=C3C=C2','O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(OC)C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC(C=CC=N3)=C3C=C2','O=C(NNC1=C(Cl)C=CC=C1Cl)C2=CC=C(C)C=C2',
             'O=C(NNC1=C(Br)C=C(Br)C=C1Br)C2=CC=C(OCC)C=C2','O=C(NNC1=C(Br)C=C(Br)C=C1Br)C2=CC=C(CC)C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(N)C=C2','O=C(NNC1=C(Br)C=CC=C1Br)C2=CC(OCO3)=C3C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(Br)C=C2','O=C(NNC1=C(F)C=CC=C1F)C2=CC=C(OCC)C=C2',
             'O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=CC=C2','O=C(NNC1=C(Br)C=CC=C1Br)C2=CC=C(F)C=C2']
molSmiles2=['O=C(NNC1=C(Cl)C=CC=C1Cl)C2=CC=C(CCC)C=C2']
mols = [Chem.MolFromSmiles(smi) for smi in molSmiles]
res = []
n=0
for i,mol in enumerate(mols):
  #print(i)
  boundsMat = rdDistGeom.GetMoleculeBoundsMatrix(mol)
  canMatch,allMatches = EmbedLib.MatchPharmacophoreToMol(mol,featFactory,pcophore)
  #print(canMatch)
  if canMatch:
    failed,boundsMatMatched,matched,matchDetails = EmbedLib.MatchPharmacophore(allMatches,boundsMat,
                                                                               pcophore,useDownsampling=True)
    if failed:
      print("Couldn't embed molecule %d"%i)
      continue
  else:
    print("Couldn't match molecule %d"%i)
    continue
  atomMatch = [list(x.GetAtomIds()) for x in matched]
  
  try:
    molH = Chem.AddHs(mol)
    bm,embeddings,numFail = EmbedLib.EmbedPharmacophore(molH,atomMatch,pcophore,count=20)
    #print(molH)
  except ValueError:
    print ("Bounds smoothing failed for molecule %d"%i)
    continue
  res.append(TransformEmbeddings(pcophore,embeddings,atomMatch))

[12:46:06] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:07] INFO: Embed failed

[12:46:08] INFO: Embed failed

[12:46:08] INFO: Embed failed

[12:46:08] INFO: Embed failed

[12:46:08] INFO: Embed failed

[12:46:08] INFO: Embed failed

[12:46:08] INFO: Embed failed

[12:46:08] INFO: Embed failed

[12:46:08] INFO: Embed failed

[12:46:08] INFO: Embed failed

[12:46:0

[12:46:16] INFO: Embed failed

[12:46:16] INFO: Embed failed

[12:46:16] INFO: Embed failed

[12:46:16] INFO: Embed failed

[12:46:16] INFO: Embed failed

[12:46:16] INFO: Embed failed

[12:46:16] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:17] INFO: Embed failed

[12:46:1

Couldn't embed molecule 27


[12:46:19] INFO: Embed failed

[12:46:19] INFO: Embed failed

[12:46:19] INFO: Embed failed

[12:46:19] INFO: Embed failed

[12:46:19] INFO: Embed failed



In [10]:
name='all_hits3.sdf'
if os.path.exists(name):
    os.remove(name)

In [11]:
ls=[]
sdfile=open(name,'a')
#mol = Chem.RWMol(Chem.Mol())
w = Chem.SDWriter(sdfile)
for i in res:
    if i is not None:
       w.write(i.mol)
w.close()
sdfile.close()

In [12]:
ls1=[]
#w = Chem.SDWriter(sdfile)
for i in res:
    if i is not None:
       ls1.append(i.score)
ls1

[5.296226710761417,
 2.8774004241503235,
 5.751557664635882,
 5.143887963178869,
 5.325391092741462,
 6.336897623644717,
 7.104849480871508,
 5.229638363934043,
 5.340195159152628,
 7.15036239549503,
 8.744204276173534,
 4.835049342517927,
 7.234291140782375,
 6.99002530343418,
 4.0332008705190106,
 3.7671388439663076,
 5.582375092831292,
 5.632229071248865,
 5.904982582393728,
 6.206535639717497,
 11.062608588394525,
 6.572290724269152,
 7.563349498566097,
 10.628024482305442,
 7.162642540370371,
 2.790471924648074,
 7.6649995947837795,
 2.8267414902118233,
 2.4648225861954067]

In [13]:
len(res)

29